<center>
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Slack Community</a>
    </p>
</center>

# **Arize Agent Mastry Course: RAG & Agentic RAG**

In the previous lab, we explored tools in depth and saw how enhancing them can strengthen our agents’ responses. Another powerful way to improve performance is by using **Retrieval-Augmented Generation (RAG)** to give the agent access to specific data sources. In this lab, the agent will retrieve relevant documents from a vector database and use that information to answer queries. We’ll continue building on the agent we created earlier.

# Set Up

In [5]:
from labs.lab1and2_base_agent import interests

from labs.lab4_tools import destination
!pip install -qqqqqqqq arize-otel agno openai openinference-instrumentation-agno openinference-instrumentation-openai httpx chromadb sentence-transformers

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [ ]:
# import os
# from getpass import getpass
#
# os.environ["ARIZE_SPACE_ID"] = globals().get("ARIZE_SPACE_ID") or getpass("🔑 Enter your Arize Space ID: ")
#
# os.environ["ARIZE_API_KEY"] = globals().get("ARIZE_API_KEY") or getpass("🔑 Enter your Arize API Key: ")
#
# os.environ["OPENAI_API_KEY"] = globals().get("OPENAI_API_KEY") or getpass("🔑 Enter your OpenAI API Key: ")
#
# os.environ["TAVILY_API_KEY"] = globals().get("TAVILY_API_KEY") or getpass("🔑 Enter your Tavily API Key: ")

In [6]:
OPENAI_API_KEY="k-proj-_nS-GmcG9QDvjU3zVmRtBpGVDFj5oh3ijinzHyCNLhNiBpu-6xBsKIylI4TcYsrArtb7v5RUKmT3BlbkFJtdVGae3sJN2aol43xpmy3gZJViFiXEn8iL1AvjrM8OedEhKCtMQIpKYRZB1e_BMtdiQ-TJF5MA"
# Or use OpenRouter (OpenAI-compatible)
# OPENROUTER_API_KEY=your_openrouter_api_key_here
# OPENROUTER_MODEL=openai/gpt-4o-mini

# Observability with Arize (https://app.arize.com)
ARIZE_SPACE_ID="U3BhY2U6NjQwOnpLb2k="
ARIZE_API_KEY="ak-8f30dd6d-75f7-4aa7-9e02-2dfe410d7693-30xLYLjC3k7TPvOd_qHulh9ut3Ga6wyo"

In [7]:
import os
import httpx
from dotenv import load_dotenv
from getpass import getpass

os.environ["ARIZE_SPACE_ID"] = ARIZE_SPACE_ID
os.environ["ARIZE_API_KEY"] = ARIZE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from arize.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.agno import AgnoInstrumentor

model_id = "travel-agent-demo"
tracer_provider = register(
    space_id=os.getenv("ARIZE_SPACE_ID"),
    api_key=os.getenv("ARIZE_API_KEY"),
    project_name=model_id,
    set_global_tracer_provider=True
)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
AgnoInstrumentor().instrument(tracer_provider=tracer_provider)

# Define Tools

The tool implementation for `essential_info` and `budget_basics` is unchanged.

In [ ]:
# --- Helper functions for tools ---
import httpx
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

@tracer.chain(name="search-api")
def _search_api(query: str) -> str | None:
    """Try Tavily search first, fall back to None."""
    tavily_key = os.getenv("TAVILY_API_KEY")
    if not tavily_key:
        return None
    try:
        resp = httpx.post(
            "https://api.tavily.com/search",
            json={
                "api_key": tavily_key,
                "query": query,
                "max_results": 3,
                "search_depth": "basic",
                "include_answer": True,
            },
            timeout=8,
        )
        data = resp.json()
        answer = data.get("answer") or ""
        snippets = [r.get("content", "") for r in data.get("results", [])]
        combined = " ".join([answer] + snippets).strip()
        return combined[:400] if combined else None
    except Exception:
        return None

def _compact(text: str, limit: int = 200) -> str:
    """Compact text for cleaner outputs."""
    cleaned = " ".join(text.split())
    return cleaned if len(cleaned) <= limit else cleaned[:limit].rsplit(" ", 1)[0]


In [ ]:
# APIs for Essential Info Tool
import httpx
from urllib.parse import quote
from typing import Optional

@tracer.chain(name="wiki-summary-api")
def _wiki_summary(dest: str) -> str:
    if not dest:
        return ""
    encoded_dest = quote(dest)

    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{encoded_dest}"
    HEADERS = { 'User-Agent': 'MyArizeApp/1.0 (ExampleContac@example.com)'}

    try:
        r = httpx.get(url, headers = HEADERS, timeout=5)
        r.raise_for_status()

        data = r.json().get("extract")
        return data if data else ""

    except httpx.HTTPStatusError as e:
        if e.response.status_code == 404:
            return ""
        return ""
    except httpx.RequestError as e:
        return ""
    except Exception as e:
        return ""

@tracer.chain(name="weather-api")
def _weather(dest):
    g = httpx.get(f"https://geocoding-api.open-meteo.com/v1/search?name={dest}")
    if g.status_code != 200 or not g.json().get("results"):
        return ""
    lat, lon = g.json()["results"][0]["latitude"], g.json()["results"][0]["longitude"]
    w = httpx.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true").json()
    cw = w.get("current_weather", {})
    return f"Weather now: {cw.get('temperature')}°C, wind {cw.get('windspeed')} km/h."

In [ ]:
from agno.tools import tool

@tool
def essential_info(destination: str) -> str:
    """Get essential info (summary and weather) using APIs"""
    parts = []
    wiki = _wiki_summary(destination)
    if wiki: parts.append(wiki)
    weather = _weather(destination)
    if weather: parts.append(weather)
    return f"{destination} essentials:\n" + "\n".join(parts)

@tool
def budget_basics(destination: str, duration: str) -> str:
    """Summarize travel cost categories."""
    q = f"{destination} travel budget average daily costs {duration}"
    s = _search_api(q)
    if s:
        return f"{destination} budget ({duration}): {_compact(s)}"
    return f"Budget for {duration} in {destination} depends on lodging, meals, transport, and attractions."

# Create RAG System for Local Flavor Tool

Now it’s time to make our `local_flavor` tool even smarter by giving it access to a rich database of travel destination insights. We’ll use ChromaDB as the vector database and a Sentence Transformer model to generate embeddings that allow the tool to find and retrieve the most relevant information.

In [8]:
import chromadb
from sentence_transformers import SentenceTransformer

chroma_client = chromadb.Client()
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create collection for local guides
collection = chroma_client.create_collection(
    name="local_guides",
    metadata={"hnsw:space": "cosine"}
)

print("✅ RAG system initialized with ChromaDB and sentence-transformers")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Newprograms\Anaconda\envs\ai\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\canut\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ RAG system initialized with ChromaDB and sentence-transformers


Download and upload `local_flavor.json` file provided to you here:

In [ ]:
# from google.colab import files
# guide = files.upload()

In [11]:
import json
with open('local_guides.json', 'r', encoding="utf8") as f:
    guides = json.load(f)

    # Prepare data for ChromaDB
documents = []
metadatas = []
ids = []

for i, guide in enumerate(guides):
    # Create a rich text representation for embedding
    text = f"City: {guide['city']}. Interests: {', '.join(guide['interests'])}. Experience: {guide['description']}"

    documents.append(text)
    metadatas.append({
      "city": guide["city"],
      "interests": ", ".join(guide["interests"]),  # ✅ make it a string
      "source": guide["source"],
      "description": guide["description"]
    })
    ids.append(f"guide_{i}")

print(documents)
print(metadatas)
print(ids)


["City: Prague. Interests: history, architecture. Experience: Join a historian for a dawn walk along the Royal Route, stepping inside St. Nicholas Church before it opens and tracing hushed Baroque courtyards up to Prague Castle for a private viewing of St. Vitus Cathedral's stained glass.", "City: Prague. Interests: food, beer. Experience: Take the Taste of Prague Vinohrady safari where guides pair farmhouse koláče with Moravian orange wine, then end in Karlín's Veltlin cellar for a vertical tasting of Czech pét-nats and tankova beer straight from the lagering tank.", "City: Prague. Interests: art, families. Experience: Spend the morning exploring DOX Centre's interactive installations before a storytelling workshop aboard the Gulliver airship, followed by hands-on puppet carving at the Museum of Czech Puppets in Malá Strana.", 'City: Prague. Interests: music, classical. Experience: Book a behind-the-scenes rehearsal at the Rudolfinum with the Czech Philharmonic, then attend an evening

In [12]:
collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )

print(f"✅ Indexed {len(documents)} experiences in vector database")

C:\Users\canut\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:11<00:00, 7.20MiB/s]  


✅ Indexed 90 experiences in vector database


In [ ]:
import json

def load_and_index_guides():

    with open('local_guides.json', 'r', encoding="utf8") as f:
      guides = json.load(f)

    # Prepare data for ChromaDB
    documents = []
    metadatas = []
    ids = []

    for i, guide in enumerate(guides):
        # Create a rich text representation for embedding
        text = f"City: {guide['city']}. Interests: {', '.join(guide['interests'])}. Experience: {guide['description']}"

        documents.append(text)
        metadatas.append({
          "city": guide["city"],
          "interests": ", ".join(guide["interests"]),  # ✅ make it a string
          "source": guide["source"],
          "description": guide["description"]
        })
        ids.append(f"guide_{i}")

    # Add to ChromaDB collection
    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )

    print(f"✅ Indexed {len(documents)} experiences in vector database")
    return len(documents)

# Load the data
num_guides = load_and_index_guides()


In [16]:
from sentence_transformers import SentenceTransformer
from openinference.semconv.trace import SpanAttributes, DocumentAttributes

# Initialize embedding model (same one you used for indexing)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
destination = 'Dubai'
interests = 'food'

query_text = f"{destination} {interests} authentic experiences"

# Embed the query
query_embedding = embedding_model.encode([query_text])

# Search in Chroma collection
results = collection.query(
  query_embeddings=query_embedding,
  n_results=3  # how many guides to retrieve
)

In [18]:
retrieved_docs = results["documents"][0]
retrieved_meta = results["metadatas"][0]

# Format a nice summary
suggestions = []
for doc, meta in zip(retrieved_docs, retrieved_meta):
    suggestion = f"📍 **{meta['city']}** — {meta['description']} (Interests: {meta['interests']})"
    suggestions.append(suggestion)

print(suggestions)
# Combine into one readable response
response = f"Here are some authentic {interests} experiences near {destination}:\n\n" + "\n\n".join(suggestions)

print(response)

['📍 **Dubai** — Follow Frying Pan Adventures through Deira for Emirati breakfast, Iranian sweet shops, and Yemeni honey tastings before joining a hands-on spice blending class at the Arabian Tea House. (Interests: food, culture)', '📍 **Dubai** — Walk Al Fahidi Historical Neighbourhood with an Emirati cultural ambassador, sip Arabic coffee at the Sheikh Mohammed Centre for Cultural Understanding, and cross the creek on an abra to visit gold and spice souks. (Interests: history, heritage)', "📍 **Dubai** — Browse handpicked Middle Eastern designers at Comptoir 102, discover concept stores in Alserkal Avenue, and receive a private fitting at the couture ateliers inside Dubai Mall's Fashion Avenue. (Interests: shopping, design)"]
Here are some authentic food experiences near Dubai:

📍 **Dubai** — Follow Frying Pan Adventures through Deira for Emirati breakfast, Iranian sweet shops, and Yemeni honey tastings before joining a hands-on spice blending class at the Arabian Tea House. (Interests:

In [ ]:
from sentence_transformers import SentenceTransformer
from openinference.semconv.trace import SpanAttributes, DocumentAttributes

# Initialize embedding model (same one you used for indexing)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

@tool
def local_flavor(destination: str, interests: str = "local culture") -> str:
    """Suggest authentic local experiences using vector retrieval from Chroma."""
    with tracer.start_as_current_span(name="RAG", attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "retriever"}) as span:
      # Construct the query text
      query_text = f"{destination} {interests} authentic experiences"
      span.set_attribute(SpanAttributes.INPUT_VALUE, query_text)

      # Embed the query
      query_embedding = embedding_model.encode([query_text])

      # Search in Chroma collection
      results = collection.query(
          query_embeddings=query_embedding,
          n_results=3  # how many guides to retrieve
      )

      # If nothing found
      if not results or not results.get("documents"):
          return f"Explore {destination}'s unique {interests} through markets, neighborhoods, and local eateries."

      # Extract retrieved guides
      retrieved_docs = results["documents"][0]
      retrieved_meta = results["metadatas"][0]
      for i, doc in enumerate(retrieved_docs):
        span.set_attribute(f"retrieval.documents.{i}.document.id", f"doc_{i}")
        span.set_attribute(f"retrieval.documents.{i}.document.content", doc)

      # Format a nice summary
      suggestions = []
      for doc, meta in zip(retrieved_docs, retrieved_meta):
          suggestion = f"📍 **{meta['city']}** — {meta['description']} (Interests: {meta['interests']})"
          suggestions.append(suggestion)

      # Combine into one readable response
      response = f"Here are some authentic {interests} experiences near {destination}:\n\n" + "\n\n".join(suggestions)
      span.set_attribute(SpanAttributes.OUTPUT_VALUE, response)

      return response


# Define Agent

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# --- Main Agent ---
trip_agent = Agent(
    name="TripPlanner",
    role="AI Travel Assistant",
    model=OpenAIChat(id="gpt-4.1"),
    instructions=(
        "You are a friendly and knowledgeable travel planner. "
        "Combine multiple tools to create a trip plan including essentials, budget, and local flavor. "
        "Keep the tone natural, clear, and under 1000 words."
    ),
    markdown=True,
    tools=[essential_info, budget_basics, local_flavor],
)

In [ ]:
# --- Run the Agent ---
destination = "Dubai"
duration = "5 days"
interests = "history, wellness"

query = f"""
Plan a {duration} trip to {destination}.
Focus on {interests}.
Include essential info, budget breakdown, and local experiences.
"""
trip_agent.print_response(
  query,
  stream=True
)

Now, if we inspect the trace, we can see that the `local_flavor` tool retrieves documents from the vector database. These retrieved documents are then used to generate tailored local recommendations.

![RAG](https://storage.googleapis.com/arize-phoenix-assets/assets/images/arize-course-rag-lab.png)